## GRACE/GRACE-FO Spatial Map Program

This notebook uses standard Python tools to demonstrate some basic visualization of the Gravity Recovery and Climate Experiment (GRACE) and the GRACE Follow-On (GRACE-FO) Level-2 spherical harmonic products.

The primary and secondary instrumentation onboard the GRACE/GRACE-FO satellites are the ranging instrument (GRACE has a microwave ranging instrument, GRACE-FO has both a microwave ranging instrument and a laser interferometer), the global positioning system (GPS), the accelerometers and the star cameras.
Data from these instruments are combined to estimate the distance between the two satellites, the positions of the satellites in space, the pointing vector of the satellites and any non-gravitational accelerations the satellites experience.
These measurements combined with background gravity field estimates, atmospheric and oceanic variability estimates and tidal estimates are used to create the [Level-2 spherical harmonic product of GRACE and GRACE-FO](https://podaac-tools.jpl.nasa.gov/drive/files/GeodeticsGravity/gracefo/docs/GRACE-FO_L2-UserHandbook_v1.1.pdf).  

There are three main processing centers that create the Level-2 spherical harmonic data as part of the GRACE/GRACE-FO Science Data System (SDS): the [University of Texas Center for Space Research (CSR)](http://www2.csr.utexas.edu/grace/), the [German Research Centre for Geosciences (GeoForschungsZentrum, GFZ)](https://www.gfz-potsdam.de/en/grace/) and the [Jet Propulsion Laboratory (JPL)](https://grace.jpl.nasa.gov/).  

The data is freely available in the US from the [NASA Physical Oceanography Distributed Active Archive Center (PO.DAAC)](https://podaac.jpl.nasa.gov/grace) and internationally from the [GFZ Information System and Data Center](http://isdc.gfz-potsdam.de/grace-isdc/).
There are programs within this repository that can sync with both of these data archives `podaac_grace_sync.py` and `gfz_isdc_grace_ftp.py`.  

This notebook uses Jupyter widgets to set parameters for converting the GRACE/GRACE-FO Level-2 product into monthly spatial maps.
The widgets can be installed as described below.  
```bash
pip3 install --user ipywidgets
jupyter nbextension enable --py --user widgetsnbextension
jupyter-notebook
```

Some of the text included within this notebook about the data processing details comes from [Sutterley and Velicogna (2019)](https://doi.org/10.3390/rs11182108) and [Sutterley et al. (2020)](https://doi.org/10.1029/2019EA000860).

### Load necessary modules for running the notebook

In [ ]:
import os
import numpy as np
import matplotlib
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams["animation.embed_limit"] = 40
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import ipywidgets
from IPython.display import HTML, Latex

import gravity_toolkit.tools
import gravity_toolkit.utilities
from gravity_toolkit.grace_input_months import grace_input_months
from gravity_toolkit.read_GIA_model import read_GIA_model
from gravity_toolkit.harmonics import harmonics
from gravity_toolkit.spatial import spatial
from gravity_toolkit.read_love_numbers import read_love_numbers
from gravity_toolkit.units import units
from gravity_toolkit.plm_holmes import plm_holmes
from gravity_toolkit.gauss_weights import gauss_weights
from gravity_toolkit.ocean_stokes import ocean_stokes
from gravity_toolkit.harmonic_summation import harmonic_summation
from gravity_toolkit.tsregress import tsregress
from gravity_toolkit.time import grace_to_calendar

### Set the GRACE/GRACE-FO Data Directory
Directory should contain:  
- Technical notes with SLR coefficients  
- Subdirectory with geocenter coefficients  
- Subdirectories for each processing center  

In [ ]:
# set the directory with GRACE/GRACE-FO data
# update local data with PO.DAAC https servers
widgets = gravity_toolkit.tools.widgets()
ipywidgets.VBox([
    widgets.directory,
    widgets.update
])

### Update Data in Directory

In [ ]:
# if updating the local data
if widgets.update.value:
    # run podaac sync program to get latest data
    !podaac_grace_sync.py --directory=$widgets.base_directory
    # run GRACE date program to verify months
    !run_grace_date.py --directory=$widgets.base_directory --verbose
    # get geocenter data from Sutterley and Velicogna (2019)
    gravity_toolkit.utilities.from_figshare(widgets.base_directory)

### Set GRACE/GRACE-FO Parameters
These parameters describe the specific GRACE/GRACE-FO product and the months of data to read  

- GRACE/GRACE-FO Processing Center
    * CSR: University of Texas Center for Space Research  
    * GFZ: German Research Centre for Geosciences (GeoForschungsZentrum)
    * JPL: Jet Propulsion Laboratory    
    * CNES: French Centre National D'Etudes Spatiales
- GRACE/GRACE-FO Data Release
- GRACE/GRACE-FO Data Product
    * GAA: non-tidal atmospheric correction  
    * GAB: non-tidal oceanic correction  
    * GAC: combined non-tidal atmospheric and oceanic correction  
    * GAD: GRACE/GRACE-FO ocean bottom pressure product  
    * GSM: corrected monthly GRACE/GRACE-FO static field product
- GRACE/GRACE-FO Date Range

In [ ]:
# update widgets
widgets.select_product()
# display widgets for setting GRACE/GRACE-FO parameters
ipywidgets.VBox([
    widgets.center,
    widgets.release,
    widgets.product,
    widgets.months
])

### Set Parameters for Reading GRACE/GRACE-FO Data
These parameters describe processing steps and corrections to be applied when reading the GRACE/GRACE-FO data

- Maximum Degree and Order
- Geocenter product (Degree 1)
- Oblateness product (<i>C</i><sub>20</sub>)
- Figure axis product (<i>C</i><sub>21</sub> and <i>S</i><sub>21</sub>)
- Azimuthal dependence product (<i>C</i><sub>22</sub> and <i>S</i><sub>22</sub>)
- Low Degree Zonal products (<i>C</i><sub>30</sub>, <i>C</i><sub>40</sub> and <i>C</i><sub>50</sub>)
- Pole Tide Correction from [Wahr et al. (2015)](https://doi.org/10.1002/2015JB011986)  
- Atmospheric Correction as described in [Fagiolini et al. (2015)](https://doi.org/10.1093/gji/ggv276)  

#### Geocenter
Measurements of time-variable gravity from the Gravity Recovery and Climate Experiment (GRACE) and the GRACE Follow-On (GRACE-FO) missions are set in a center of mass (CM) reference frame, in which the total degree one variations are inherently zero.
The individual contributions to degree one variations in the CM reference frame, such as from oceanic processes or terrestrial water storage change, are not necessarily zero ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
Applications set in a center of figure (CF) reference frame, such as the recovery of mass variations of the oceans, hydrosphere and cryosphere, require the inclusion of degree one terms to be fully accurate ([Swenson et al., 2008](https://doi.org/10.1029/2007JB005338)).
The geocenter options above can select the degree one product to include with the GRACE/GRACE-FO derived harmonics of degree two and higher.
There are options for using measurements from satellite laser ranging ([Cheng, 2013](https://doi.org/10.1007/978-3-642-32998-2_4)) and calculations from time-variable gravity and ocean model outputs ([Swenson et al., 2008](https://doi.org/10.1029/2007JB005338)).

#### Low Degree Harmonics
For both GRACE and GRACE-FO, there have been operational issues that have affected the quality of the time-variable gravity fields.
During the late stages of the GRACE mission, procedures were enacted to preserve the battery life of the GRACE satellites and extend the mission lifetime.
This included turning off the accelerometer and microwave ranging instrument (MWI) to reduce the battery load on the spacecraft during periods of low &beta; angle when solar input to the spacecraft is lowest.
In late 2016, the accelerometer onboard GRACE-B was permanently powered down to help maintain the operation of the ranging instrument.
For the Follow-On mission, an anomaly shortly after the launch of GRACE-FO caused a malfunction and the accelerometer onboard GRACE-FO 2 (GF2) has been operating in a less optimal Large-Range-Mode.
For both of these cases, the GRACE/GRACE-FO processing centers have developed independent methods to spatiotemporally transplant the accelerometer data retrieved from GRACE-A to GRACE-B and from GF1 to GF2 ([Bandikova et al., 2019](https://doi.org/10.1016/j.asr.2019.05.021)).
These single-accelerometer months for both GRACE and GRACE-FO contain significantly more noise, particularly the low-degree zonal harmonics (predominantly <i>C</i><sub>20</sub> and <i>C</i><sub>30</sub> but possibly <i>C</i><sub>40</sub> and <i>C</i><sub>50</sub>).
<i>C</i><sub>20</sub> has also been difficult for GRACE and GRACE-FO to independently measure throughout both missions.
The figure axis harmonics (<i>C</i><sub>21</sub> and <i>S</i><sub>21</sub>) may also be contaminated
by noise during the single-accelerometer months in the GFZ products [Dahle et al, 2019](https://doi.org/10.3390/rs11182116).
Measurements from satellite laser ranging (SLR) can provide an independent assessment for some low degree and order spherical harmonics ([Cheng et al., 2011](https://doi.org/10.1029/2010JB000850); [Cheng and Ries, 2018](https://doi.org/10.1093/gji/ggx483); [Loomis et al., 2019](https://doi.org/10.1029/2019GL082929); [Loomis et al., 2020](https://doi.org/10.1029/2019GL085488)).
The <i>C</i><sub>20</sub>, <i>C</i><sub>21</sub>, <i>C</i><sub>22</sub>, <i>C</i><sub>30</sub>, <i>C</i><sub>40</sub>, and <i>C</i><sub>50</sub> options below can select the SLR low degree product to use as a replacement for the GRACE/GRACE-FO derived estimates.

#### Corrections
Prior to GRACE/GRACE-FO Release-6, corrections needed to be applied to compensate for long-period signals in the pole tide that were contaminating the <i>C</i><sub>21</sub> and <i>S</i><sub>21</sub> harmonics ([Wahr et al., 2015](https://doi.org/10.1002/2015JB011986)), as well as for discontinuities in the atmospheric de-aliasing product that were introduced with upgrades in the ECMWF weather prediction model ([Fagiolini et al., 2015](https://doi.org/10.1093/gji/ggv276)).  The Pole Tide and Atmospheric corrections do not need to be applied to the Release-6 data. 

In [ ]:
# update widgets
widgets.select_options()
# display widgets for setting GRACE/GRACE-FO read parameters
ipywidgets.VBox([
    widgets.lmax,
    widgets.mmax,
    widgets.geocenter,
    widgets.C20,
    widgets.CS21,
    widgets.CS22,
    widgets.C30,
    widgets.C40,
    widgets.C50,
    widgets.pole_tide,
    widgets.atm
])

### Read GRACE/GRACE-FO data
This step extracts the parameters chosen above and then reads the GRACE/GRACE-FO data applying the specified procedures  

In [ ]:
# extract values from widgets
PROC = widgets.center.value
DREL = widgets.release.value
DSET = widgets.product.value
months = [int(m) for m in widgets.months.value]
LMAX = widgets.lmax.value
MMAX = widgets.mmax.value
DEG1 = widgets.geocenter.value
SLR_C20 = widgets.C20.value
SLR_21 = widgets.CS21.value
SLR_22 = widgets.CS22.value
SLR_C30 = widgets.C30.value
SLR_C40 = widgets.C40.value
SLR_C50 = widgets.C50.value
POLE_TIDE = widgets.pole_tide.value
ATM = widgets.atm.value

# read GRACE/GRACE-FO data for parameters
start_mon = np.min(months)
end_mon = np.max(months)
missing = sorted(set(np.arange(start_mon,end_mon+1)) - set(months))
Ylms = grace_input_months(widgets.base_directory, PROC, DREL, DSET,
    LMAX, start_mon, end_mon, missing, SLR_C20, DEG1, MMAX=MMAX,
    SLR_21=SLR_21, SLR_22=SLR_22, SLR_C30=SLR_C30, SLR_C40=SLR_C40,
    SLR_C50=SLR_C50, POLE_TIDE=POLE_TIDE, ATM=ATM)
# create harmonics object and remove mean
GRACE_Ylms = harmonics().from_dict(Ylms)
GRACE_Ylms.mean(apply=True)
# directory of specific GRACE/GRACE-FO product
GRACE_Ylms.directory = Ylms['directory']
# string denoting specific corrections and data used
GRACE_Ylms.title = Ylms['title']
# number of time steps
nt = len(months)
# flag for spherical harmonic order
order_str = f'M{MMAX:d}' if (MMAX != LMAX) else ''

### Set Parameters to Convert to Spatial Maps
These parameters specify corrections and filtering steps for converting to the spatial domain at a specified grid spacing  

- GIA Correction  
- Remove Specific Harmonic Fields  
- Redistribute Removed Fields over the Ocean  
- Gaussian Smoothing Radius in kilometers  
- Filter (destripe) harmonics [(Swenson and Wahr, 2006)](https://doi.org/10.1029/2005GL025285)  
- Spatial degree spacing  
- Spatial degree interval  
    1) (-180:180,90:-90)  
    2) (degree spacing)/2  
- Output spatial units  
    1) equivalent water thickness (cm)  
    2) geoid height (mm)  
    3) elastic crustal deformation (mm)  
    4) gravitational perturbation (&mu;Gal)  
    5) equivalent surface pressure (mbar)  
- Output spatial format  
    1) netCDF4  
    2) HDF5  

#### Geophysical Leakage
Gravity measurements from GRACE and GRACE-FO are global, near-monthly and are directly related to changes in mass.
Several mass transport processes can occur concurrently for a given region, which means that the total time-dependent geopotential from GRACE/GRACE-FO can relate to multiple time-varying components ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
These mass transport processes include but are not limited to terrestrial water storage, glacier and ice sheet mass, atmospheric and oceanic circulation and geodynamic processes.
In order to isolate the mass change of a single process, each of the other processes needs to be independently estimated and removed from the GRACE/GRACE-FO data.
Uncertainties in the components removed from the GRACE/GRACE-FO data will directly impact the precision of the final mass balance estimate.

#### Filtering
The GRACE/GRACE-FO coefficients are impacted by random spherical harmonic errors that increase as a function of spherical harmonic degree ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
The impact of these errors can be reduced using Gaussian averaging functions as described in  [Jekeli, (1981)](http://www.geology.osu.edu/~jekeli.1/OSUReports/reports/report_327.pdf).
GRACE/GRACE-FO coefficients are also impacted by correlated north/south "striping" errors, which can be spectrally filtered following [Swenson and Wahr (2006)](https://doi.org/10.1029/2005GL025285).

#### Units
Spatial fields of surface mass density can be estimated from sets of spherical harmonics if we assume that the mass redistributions are concentrated within a thin layer (thickness &#x226a; horizontal resolution) after correcting for glacial isostatic adjustment ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
However, we additionally need to compensate for the Earth's elastic yielding to surface load changes, which induce density anomalies at depth ([Wahr et al., 1998](https://doi.org/10.1029/98JB02844)).
This program accounts for the elastic deformation of the solid Earth using load Love numbers calculated by [Han and Wahr (1995)](https://doi.org/10.1111/j.1365-246X.1995.tb01819.x) with parameters from the Preliminary reference Earth model (PREM).
This program can output monthly spatial fields in a few different units (mass, deformation, and gravity).
- Equivalent water thickness
    $$\Delta\sigma(\theta,\phi) = \frac{a\rho_{e}}{3\rho_{w}}\sum_{l=1}^{l_{max}}\sum_{m=0}^l\frac{2l+1}{1+k_l}\tilde{P}_{lm}(\cos\theta)\left[\Delta \tilde{C}_{lm}\cos{m\phi}+\Delta \tilde{S}_{lm}\sin{m\phi}\right]$$
- Geoid height
    $$\Delta N(\theta,\phi) = a\sum_{l=1}^{l_{max}}\sum_{m=0}^l \tilde{P}_{lm}(\cos\theta)\left[\tilde{C}_{lm}\cos{m\phi}+ \tilde{S}_{lm}\sin{m\phi}\right]$$
- Vertical elastic crustal deformation  
    $$\Delta z(\theta,\phi) = a\sum_{l=1}^{l_{max}}\sum_{m=0}^l\frac{h_l}{1+k_l}\tilde{P}_{lm}(\cos\theta)\left[\Delta \tilde{C}_{lm}\cos{m\phi}+\Delta \tilde{S}_{lm}\sin{m\phi}\right]$$

    * <i>a</i>: average radius of the Earth  
    * <i>&rho;<sub>e</sub></i>: average density of the Earth  
    * <i>&rho;<sub>w</sub></i>: density of water  
    * <i>h<sub>l</sub></i>, <i>k<sub>l</sub></i>: Load Love numbers of degree <i>l</i> 
    * <i>P<sub>lm</sub></i>: Fully normalized Legendre polynomials of degree <i>l</i> and order <i>m</i> 
    * <i>C<sub>lm</sub></i>, <i>S<sub>lm</sub></i>: Cosine and sine spherical harmonics of degree <i>l</i> and order <i>m</i> 
    * <i>&theta;</i>, <i>&phi;</i>: Colatitude and longitude in radians


In [ ]:
# update widgets
widgets.select_corrections()
widgets.select_output()
# display widgets for setting GRACE/GRACE-FO corrections parameters
ipywidgets.VBox([
    widgets.GIA_file,
    widgets.GIA,
    widgets.remove_file,
    widgets.remove_format,
    widgets.redistribute_removed,
    widgets.mask,
    widgets.gaussian,
    widgets.destripe,
    widgets.spacing,
    widgets.interval,
    widgets.units,
    widgets.output_format])

### Convert GRACE/GRACE-FO harmonics to the spatial domain
This step extracts the parameters chosen above and then converts the GRACE/GRACE-FO harmonics to the spatial domain applying the specified corrections and filtering procedures

- Set output grid domain  
- Calculate Fully-Normalized Legendre Polynomials  
- Read GIA model for correcting GRACE/GRACE-FO data  
- Read harmonics to be removed from the GRACE/GRACE-FO data  
- Calculate coefficients for converting to the output units  
- Convert from the spherical harmonic domain into the spatial domain  
- Output the monthly spatial maps to netCDF4 or HDF5  

In [ ]:
# Output spatial data
grid = spatial()
grid.time = np.copy(GRACE_Ylms.time)
grid.month = np.copy(GRACE_Ylms.month)

# Output degree spacing
dlon = widgets.spacing.value
dlat = widgets.spacing.value
# Output Degree Interval
INTERVAL = widgets.interval.index + 1
if (INTERVAL == 1):
    # (-180:180,90:-90)
    nlon = np.int64((360.0/dlon)+1.0)
    nlat = np.int64((180.0/dlat)+1.0)
    grid.lon = -180 + dlon*np.arange(0,nlon)
    grid.lat = 90.0 - dlat*np.arange(0,nlat)
elif (INTERVAL == 2):
    # (Degree spacing)/2
    grid.lon = np.arange(-180+dlon/2.0,180+dlon/2.0,dlon)
    grid.lat = np.arange(90.0-dlat/2.0,-90.0-dlat/2.0,-dlat)
    nlon = len(grid.lon)
    nlat = len(grid.lat)
# update spacing and dimensions
grid.update_spacing()
grid.update_extents()
grid.update_dimensions()

# Computing plms for converting to spatial domain
theta = (90.0-grid.lat)*np.pi/180.0
PLM, dPLM = plm_holmes(LMAX, np.cos(theta))

# read load love numbers file
# PREM outputs from Han and Wahr (1995)
# https://doi.org/10.1111/j.1365-246X.1995.tb01819.x
love_numbers_file = gravity_toolkit.utilities.get_data_path(['data','love_numbers'])
header = 2
columns = ['l','hl','kl','ll']
# LMAX of load love numbers from Han and Wahr (1995) is 696.
# from Wahr (2007) linearly interpolating kl works
# however, as we are linearly extrapolating out, do not make
# LMAX too much larger than 696
# read arrays of kl, hl, and ll Love Numbers
hl,kl,ll = read_love_numbers(love_numbers_file, LMAX=LMAX, HEADER=header,
    COLUMNS=columns, REFERENCE='CF', FORMAT='tuple')

# read GIA data
GIA = widgets.GIA.value
GIA_Ylms_rate = read_GIA_model(widgets.GIA_model,
    GIA=GIA, LMAX=LMAX, MMAX=MMAX)
gia_str = '' if (GIA == '[None]') else '_{0}'.format(GIA_Ylms_rate['title'])
# calculate the monthly mass change from GIA
GIA_Ylms = GRACE_Ylms.zeros_like()
GIA_Ylms.time[:] = np.copy(GRACE_Ylms.time)
GIA_Ylms.month[:] = np.copy(GRACE_Ylms.month)
# monthly GIA calculated by gia_rate*time elapsed
# finding change in GIA each month
for t in range(nt):
    GIA_Ylms.clm[:,:,t] = GIA_Ylms_rate['clm']*(GIA_Ylms.time[t]-2003.3)
    GIA_Ylms.slm[:,:,t] = GIA_Ylms_rate['slm']*(GIA_Ylms.time[t]-2003.3)

# if redistributing removed mass over the ocean
if widgets.redistribute_removed.value:
    # read Land-Sea Mask and convert to spherical harmonics
    ocean_Ylms = ocean_stokes(widgets.landmask, LMAX,
        MMAX=MMAX, LOVE=(hl,kl,ll))
    
# read data to be removed from GRACE/GRACE-FO monthly harmonics
remove_Ylms = GRACE_Ylms.zeros_like()
remove_Ylms.time[:] = np.copy(GRACE_Ylms.time)
remove_Ylms.month[:] = np.copy(GRACE_Ylms.time)
# If there are files to be removed from the GRACE/GRACE-FO data
# for each file separated by commas
for f in widgets.remove_files:
    if (widgets.remove_format.value == 'netCDF4'):
        # read netCDF4 file
        Ylms = harmonics().from_netCDF4(f)
    elif (widgets.remove_format.value == 'HDF5'):
        # read HDF5 file
        Ylms = harmonics().from_HDF5(f)
    elif (widgets.remove_format.value == 'index (ascii)'):
        # read index of ascii files
        Ylms = harmonics().from_index(f,format='ascii')
    elif (widgets.remove_format.value == 'index (netCDF4)'):
        # read index of netCDF4 files
        Ylms = harmonics().from_index(f,format='netCDF4')
    elif (widgets.remove_format.value == 'index (HDF5)'):
        # read index of HDF5 files
        Ylms = harmonics().from_index(f,format='HDF5')
    # reduce to months of interest and truncate to range
    Ylms = Ylms.subset(months).truncate(LMAX,mmax=MMAX)
    # redistribute removed mass over the ocean
    if widgets.redistribute_removed.value:
        # calculate ratio between total removed mass and
        # a uniformly distributed cm of water over the ocean
        ratio = Ylms.clm[0,0,:]/ocean_Ylms.clm[0,0]
        # for each spherical harmonic
        for m in range(0,MMAX+1):
            for l in range(m,LMAX+1):
                # remove the ratio*ocean Ylms from Ylms
                Ylms.clm[l,m,:]-=ratio*ocean_Ylms.clm[l,m]
                Ylms.slm[l,m,:]-=ratio*ocean_Ylms.slm[l,m]
    # add the harmonics to be removed to the total
    remove_Ylms.add(Ylms)

# gaussian smoothing radius in km (Jekeli, 1981)
RAD = widgets.gaussian.value
if (RAD != 0):
    wt = 2.0*np.pi*gauss_weights(RAD,LMAX)
    gw_str = f'_r{RAD:0.0f}km'
else:
    # else = 1
    wt = np.ones((LMAX+1))
    gw_str = ''

# destriping the GRACE/GRACE-FO harmonics
ds_str = '_FL' if widgets.destripe.value else ''

# Setting units factor for output
UNITS = widgets.unit_index
# dfactor is the degree dependent coefficients
# for specific spherical harmonic output units
if (UNITS == 1):
    # 1: cmwe, centimeters water equivalent
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).cmwe
elif (UNITS == 2):
    # 2: mmGH, millimeters geoid height
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mmGH
elif (UNITS == 3):
    # 3: mmCU, millimeters elastic crustal deformation
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mmCU
elif (UNITS == 4):
    # 4: micGal, microGal gravity perturbations
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).microGal
elif (UNITS == 5):
    # 5: mbar, millibar equivalent surface pressure
    dfactor = units(lmax=LMAX).harmonic(hl,kl,ll).mbar
# units strings for output files and plots
unit_label = ['cm', 'mm', 'mm', u'\u03BCGal', 'mb']
unit_name = ['Equivalent Water Thickness', 'Geoid Height',
    'Elastic Crustal Uplift', 'Gravitational Undulation',
    'Equivalent Surface Pressure']

# converting harmonics to truncated, smoothed coefficients in units
# combining harmonics to calculate output spatial fields
# output spatial grid
grid.data = np.zeros((nlat,nlon,nt))
grid.mask = np.zeros((nlat,nlon,nt), dtype=bool)
for i,grace_month in enumerate(GRACE_Ylms.month):
    # GRACE/GRACE-FO harmonics for time t
    # and monthly files to be removed
    if widgets.destripe.value:
        Ylms = GRACE_Ylms.index(i).destripe()
        Ylms.subtract(remove_Ylms.index(i).destripe())
    else:
        Ylms = GRACE_Ylms.index(i)
        Ylms.subtract(remove_Ylms.index(i))
    # Remove GIA rate for time
    Ylms.subtract(GIA_Ylms.index(i))
    # smooth harmonics and convert to output units
    Ylms.convolve(dfactor*wt)
    # convert spherical harmonics to output spatial grid
    grid.data[:,:,i] = harmonic_summation(Ylms.clm, Ylms.slm,
        grid.lon, grid.lat, LMAX=LMAX, MMAX=MMAX, PLM=PLM).T

#### Output spatial data to file

In [ ]:
# output to netCDF4 or HDF5
suffix = dict(netCDF4='nc',HDF5='H5')
file_format = '{0}_{1}_{2}{3}{4}_{5}_L{6:d}{7}{8}{9}_{10:03d}-{11:03d}.{12}'
if widgets.format in ('netCDF4','HDF5'):
    FILE = file_format.format(PROC,DREL,DSET,gia_str,GRACE_Ylms.title,
        widgets.units.value,LMAX,order_str,gw_str,ds_str,
        months[0],months[-1],suffix[widgets.format])
    grid.to_file(os.path.join(GRACE_Ylms.directory,FILE),
        format=widgets.format, varname='z',
        units=widgets.units.value, longname=unit_name[UNITS-1],
        title='GRACE/GRACE-FO Spatial Data', date=True)


### Create animation of GRACE/GRACE-FO months

In [ ]:
# slider for the plot min and max for normalization
vmin = np.min(grid.data).astype(np.int64)
vmax = np.ceil(np.max(grid.data)).astype(np.int64)
cmap1 = gravity_toolkit.tools.colormap(vmin=vmin, vmax=vmax)
# display widgets for setting GRACE/GRACE-FO regression plot parameters
ipywidgets.VBox([cmap1.range,cmap1.step,cmap1.name,cmap1.reverse])

In [ ]:
fig, ax1 = plt.subplots(num=1, nrows=1, ncols=1, figsize=(10.375,6.625),
    subplot_kw=dict(projection=ccrs.PlateCarree()))

# levels and normalization for plot range
im = ax1.imshow(np.zeros((nlat,nlon)), interpolation='nearest',
    norm=cmap1.norm, cmap=cmap1.value, transform=ccrs.PlateCarree(),
    extent=grid.extent, origin='upper', animated=True)
ax1.coastlines('50m')

# add date label (year-calendar month e.g. 2002-01)
time_text = ax1.text(0.025, 0.025, '', transform=fig.transFigure,
    color='k', size=24, ha='left', va='baseline')

# Add horizontal colorbar and adjust size
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# pad = distance from main plot axis
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax1, extend='both', extendfrac=0.0375,
    orientation='horizontal', pad=0.025, shrink=0.85,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel(unit_name[UNITS-1], labelpad=10, fontsize=24)
cbar.ax.set_ylabel(unit_label[UNITS-1], fontsize=24, rotation=0)
cbar.ax.yaxis.set_label_coords(1.045, 0.1)
# Set the tick levels for the colorbar
cbar.set_ticks(cmap1.levels)
cbar.set_ticklabels(cmap1.label)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=26, labelsize=24,
    direction='in')
    
# stronger linewidth on frame
ax1.spines['geo'].set_linewidth(2.0)
ax1.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.patch.set_facecolor('white')
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
    
# animate frames
def animate_frames(i):
    # set image
    im.set_data(grid.data[:,:,i])
    # add date label (year-calendar month e.g. 2002-01)
    year,month = grace_to_calendar(grid.month[i])
    time_text.set_text(u'{0:4d}\u2013{1:02d}'.format(year,month))

# set animation
anim = animation.FuncAnimation(fig, animate_frames, frames=nt)
%matplotlib inline
HTML(anim.to_jshtml())

### Time Series Analysis
- Using ordinary least-squares to fit a synthetic signal to the GRACE/GRACE-FO derived spatial fields
- Can fit both polynomial and cyclical (seasonal) components
- Can be used to analyze both the seasonal and long-term change in regional mass distributions

In [ ]:
style = {'description_width': 'initial'}
orderText = ipywidgets.BoundedIntText(
    value=1,
    min=0,
    max=4,
    step=1,
    description='Polynomial Order:',
    disabled=False,
    style=style,
)

# cyclical options
cyclicLabel = ipywidgets.Label('Cyclical Terms:')
cyclicCheckbox = {}
for key in ['Annual','Semi-Annual','S2 Tide']:
    cyclicCheckbox[key] = ipywidgets.Checkbox(
        value=True,
        description=key,
        disabled=False,
    )
cyclic = ipywidgets.HBox([cyclicLabel,*cyclicCheckbox.values()])

# display widgets for setting GRACE/GRACE-FO regression parameters
ipywidgets.VBox([orderText,cyclic])

In [ ]:
# build list of cyclical components
ORDER = orderText.value
PHASES = {'Annual':1.0,'Semi-Annual':0.5,'S2 Tide':161.0/365.25}
CYCLES = [v for k,v in PHASES.items() if cyclicCheckbox[k]]
ncomp = (ORDER + 1) + 2*len(CYCLES)
# Allocating memory for output variables
out = spatial(spacing=grid.spacing, nlon=nlon, nlat=nlat,
    extent=grid.extent, fill_value=grid.fill_value)
out.data = np.zeros((nlat,nlon,ncomp))
# update mask and dimensions
out.update_dimensions()
out.update_mask()

# calculate the regression coefficients
for i in range(nlat):
    for j in range(nlon):
        # Calculating the regression coefficients
        tsbeta = tsregress(grid.time, grid.data[i,j,:],
            ORDER=ORDER, CYCLES=CYCLES)
        # save regression components
        for k in range(0, ncomp):
            out.data[i,j,k] = tsbeta['beta'][k]

### Set parameters for creating regression plot
This step specifies the regression variable to plot, the [colormap](https://matplotlib.org/gallery/color/colormap_reference.html), the normalization for the contour plot colors and the step for the color bar.


In [ ]:
# strings for polynomial terms
if (ORDER == 0):# Mean
    variable_longname = ['Mean']
elif (ORDER == 1):# Trend
    variable_longname = ['Constant','Trend']
elif (ORDER == 2):# Quadratic
    variable_longname = ['Constant','Linear','Quadratic']
unit_suffix = [' yr$^{{{0:d}}}$'.format(-o) if o else '' for o in range(ORDER+1)]
# strings for cyclical terms
cyclic_longname = {}
cyclic_longname['Annual'] = ['Annual Sine', 'Annual Cosine']
cyclic_longname['Semi-Annual'] = ['Semi-Annual Sine', 'Semi-Annual Cosine']
cyclic_longname['S2 Tide'] = ['S2 Tidal Alias Sine', 'S2 Tidal Alias Cosine']
# combined strings for all components
variable_longname.extend([i for k,v in cyclic_longname.items() for i in v if cyclicCheckbox[k]])
unit_suffix.extend(['' for k,v in cyclic_longname.items() for i in v if cyclicCheckbox[k]])

# variable of interest
variableDropdown = ipywidgets.Dropdown(
    options=variable_longname,
    value=variable_longname[ORDER],
    description='Variable:',
    disabled=False,
)

# slider for the plot min and max for normalization
i = variableDropdown.index
vmin = np.min(out.data[:,:,i]).astype(np.int64)
vmax = np.ceil(np.max(out.data[:,:,i])).astype(np.int64)
cmap2 = gravity_toolkit.tools.colormap(vmin=vmin, vmax=vmax)

# set range and step size for variable
def set_range_and_step(sender):
    i = variableDropdown.index
    cmin = np.min(out.data[:,:,i]).astype(np.int64)
    cmax = np.ceil(np.max(out.data[:,:,i])).astype(np.int64)
    cmap2.range.min = cmin
    cmap2.range.max = cmax
    cmap2.range.value = [cmin,cmax]
    cmap2.step.max = cmax - cmin

# watch variable widget for changes
variableDropdown.observe(set_range_and_step)

# display widgets for setting GRACE/GRACE-FO regression plot parameters
ipywidgets.VBox([variableDropdown,cmap2.range,cmap2.step,cmap2.name,cmap2.reverse])

### Create plot of regression outputs from GRACE/GRACE-FO data

In [ ]:
fig, ax2 = plt.subplots(num=2, nrows=1, ncols=1, figsize=(10.375,6.625),
    subplot_kw=dict(projection=ccrs.PlateCarree()))

# levels and normalization for plot range
i = variableDropdown.index
im = ax2.imshow(out.data[:,:,i], interpolation='nearest',
    norm=cmap2.norm, cmap=cmap2.value, transform=ccrs.PlateCarree(),
    extent=grid.extent, origin='upper')
ax2.coastlines('50m')

# Add horizontal colorbar and adjust size
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# pad = distance from main plot axis
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax2, extend='both', extendfrac=0.0375,
    orientation='horizontal', pad=0.025, shrink=0.85,
    aspect=22, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_xlabel('{0} [{1}{2}]'.format(unit_name[UNITS-1],
    unit_label[UNITS-1], unit_suffix[i]), labelpad=10, fontsize=24)
# Set the tick levels for the colorbar
cbar.set_ticks(cmap2.levels)
cbar.set_ticklabels(cmap2.label)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=26, labelsize=24,
    direction='in')
    
# stronger linewidth on frame
ax2.spines['geo'].set_linewidth(2.0)
ax2.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.patch.set_facecolor('white')
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
plt.show()